In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine
from sklearn.linear_model import Ridge

In [ ]:
##1 Content-based

In [2]:
data = [['a', 9, 8, 4, 2, 6],
        ['b', 6, 8, 6, 5, 5], 
        ['c', 7, 6, 7, None, 6], 
        ['d', 5, 5, 5, 6, None], 
        ['e', 8, 5, 5, 2, None],  
        ['f', 5, 6, 6, 5, 8], 
        ['g', 4, 8, 9, 7, None], 
        ['h', 7, 8, 7, None, None], 
        ['i', None, 2, 5, 6, 5],
        ['j', 5, None, None, 6, 6],
        ['k', 5, 6, 7, 6, 8], 
        ['l', 2, 5, 8, None, 6]] 
df = pd.DataFrame(data)
df.columns = ['student', 'mathematics', 'chemistry',  'physics', 'literature', 'history']
df

,student,mathematics,chemistry,physics,literature,history
0,a,9.0,8.0,4.0,2.0,6.0
1,b,6.0,8.0,6.0,5.0,5.0
2,c,7.0,6.0,7.0,NaN,6.0
3,d,5.0,5.0,5.0,6.0,NaN
4,e,8.0,5.0,5.0,2.0,NaN
5,f,5.0,6.0,6.0,5.0,8.0
6,g,4.0,8.0,9.0,7.0,NaN
7,h,7.0,8.0,7.0,NaN,NaN
8,i,NaN,2.0,5.0,6.0,5.0
9,j,5.0,NaN,NaN,6.0,6.0


In [3]:
sub = pd.DataFrame(['feature vector', [0.99, 0.01], [0.95, 0.05], [0.97, 0.03], [0.01, 0.99], [0.02, 0.98]]).T
sub.columns = df.columns
df2 = pd.concat([df, sub], ignore_index = True)
df2.set_index('student', inplace = True)
df2.fillna('None', inplace = True)
df2

,mathematics,chemistry,physics,literature,history
student,,,,,
a,9,8,4,2,6
b,6,8,6,5,5
c,7,6,7,None,6
d,5,5,5,6,None
e,8,5,5,2,None
f,5,6,6,5,8
g,4,8,9,7,None
h,7,8,7,None,None
i,None,2,5,6,5


In [4]:
sub = sub.values[0][1:].tolist()
sub

[[0.99, 0.01], [0.95, 0.05], [0.97, 0.03], [0.01, 0.99], [0.02, 0.98]]

In [5]:
df2 = df2.iloc[:-1, :]
result = list()
for i in range(df2.shape[0]):
    if 'None' in df2.iloc[i].values:
        val = df2.iloc[i].values
        idx = [k for k in range(len(val)) if val[k] != 'None']
        y_train = [val[k] for k in range(len(val)) if k in idx]
        x_train = [sub[k] for k in range(len(val)) if k in idx]
        x_test = [sub[k] for k in range(len(val)) if k not in idx]
        #print(y_train, x_test, x_train)
        model = Ridge(alpha=1.0)
        model.fit(x_train, y_train)
        y = model.predict(x_test).tolist()
        
        for j in range(len(val)):
            if val[j] == 'None':
                val[j] = round(y.pop(0), 2)
        result.append(val)
    else: result.append(df2.iloc[i].values)
result = pd.DataFrame(result)      
result.columns = df2.columns
result.index = df2.index
result

,mathematics,chemistry,physics,literature,history
student,,,,,
a,9.00,8.00,4.00,2.00,6.00
b,6.00,8.00,6.00,5.00,5.00
c,7.00,6.00,7.00,6.20,6.00
d,5.00,5.00,5.00,6.00,5.68
e,8.00,5.00,5.00,2.00,3.25
f,5.00,6.00,6.00,5.00,8.00
g,4.00,8.00,9.00,7.00,7.05
h,7.00,8.00,7.00,7.37,7.37
i,3.83,2.00,5.00,6.00,5.00


In [ ]:
##2 Collaborative Filtering

In [9]:
df3 = df.T
df3.rename(columns = df3.iloc[0], inplace = True)
df3 = df3.drop(df3.index[0])
df3


,a,b,c,d,e,f,g,h,i,j,k,l
mathematics,9,6,7,5,8,5,4,7,NaN,5,5,2
chemistry,8,8,6,5,5,6,8,8,2,NaN,6,5
physics,4,6,7,5,5,6,9,7,5,NaN,7,8
literature,2,5,NaN,6,2,5,7,NaN,6,6,6,NaN
history,6,5,6,NaN,NaN,8,NaN,NaN,5,6,8,6


In [10]:
df3.iloc[3, 2] == np.nan
mean = df3.mean(axis = 0)
pd.DataFrame(mean).T
lst1 = list() 
df3.fillna('None', inplace = True)
for i in range(df3.shape[0]):
    lst2 = list()
    for j in range(df3.shape[1]):
        if df3.iloc[i, j] == 'None':
            lst2.append(0)
        else:
            lst2.append(df3.iloc[i, j] - mean[j])
    lst1.append(lst2)
df4 = pd.DataFrame(lst1)
df4

,0,1,2,3,4,5,6,7,8,9,10,11
0,3.2,0.0,0.5,-0.25,3.0,-1.0,-3.0,-0.333333,0.0,-0.666667,-1.4,-3.25
1,2.2,2.0,-0.5,-0.25,0.0,0.0,1.0,0.666667,-2.5,0.000000,-0.4,-0.25
2,-1.8,0.0,0.5,-0.25,0.0,0.0,2.0,-0.333333,0.5,0.000000,0.6,2.75
3,-3.8,-1.0,0.0,0.75,-3.0,-1.0,0.0,0.000000,1.5,0.333333,-0.4,0.00
4,0.2,-1.0,-0.5,0.00,0.0,2.0,0.0,0.000000,0.5,0.333333,1.6,0.75


In [11]:
cosine_similarity = pd.DataFrame(1 - pairwise_distances(df4.T.values, metric = "cosine"))
cosine_similarity

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.000000,0.570266,-0.087304,-7.560730e-01,0.864263,0.071283,-0.513324,2.138497e-01,-0.698430,-0.712832,-3.522242e-01,-0.635101
1,0.570266,1.000000,-0.204124,-5.892557e-01,0.288675,-0.166667,0.218218,6.666667e-01,-0.952579,-0.333333,-3.580574e-01,-0.117851
2,-0.087304,-0.204124,1.000000,-1.443376e-01,0.353553,-0.612372,-0.267261,-8.164966e-01,0.416667,-0.612372,-4.385290e-01,-0.115470
3,-0.756073,-0.589256,-0.144338,1.000000e+00,-0.816497,-0.235702,0.000000,-2.220446e-16,0.625463,0.589256,0.000000e+00,0.050000
4,0.864263,0.288675,0.353553,-8.164966e-01,1.000000,0.000000,-0.566947,-2.886751e-01,-0.353553,-0.866025,-3.100868e-01,-0.530723
5,0.071283,-0.166667,-0.612372,-2.357023e-01,0.000000,1.000000,0.327327,1.666667e-01,-0.068041,0.500000,8.951436e-01,0.447834
6,-0.513324,0.218218,-0.267261,0.000000e+00,-0.566947,0.327327,1.000000,3.273268e-01,-0.133631,0.654654,5.860090e-01,0.925820
7,0.213850,0.666667,-0.816497,-2.220446e-16,-0.288675,0.166667,0.327327,1.000000e+00,-0.748455,0.333333,-2.220446e-16,0.000000
8,-0.698430,-0.952579,0.416667,6.254628e-01,-0.353553,-0.068041,-0.133631,-7.484552e-01,1.000000,0.272166,2.192645e-01,0.182828
9,-0.712832,-0.333333,-0.612372,5.892557e-01,-0.866025,0.500000,0.654654,3.333333e-01,0.272166,1.000000,7.161149e-01,0.683537


In [12]:
def useranduser_prediction(k, df, similarity, nor_df, index):
    df = list(df.iloc[index[1]])
    idx = [i for i in range(len(df)) if df[i] != 'None']
    nor_df = list(nor_df.iloc[index[1]])
    similarity = list(similarity.iloc[2])
    lst = list()
    for i in range(len(similarity)):
        if i not in idx: continue
        lst.append([similarity[i], nor_df[i]])
    lst.sort()
    lst = lst[-k:]
    return sum([np.prod(i) for i in lst]) / sum([abs(a) for a, b in lst])
useranduser_prediction(2, df3, cosine_similarity, df4, (2, 3))

-0.5656307800262734

In [13]:
k = 2
df4_fix = df4.copy()
for i in range(df3.shape[0]-1):
    for j in range(df3.shape[1]-1):
        if df3.iloc[i, j] == 'None':
            df4_fix.iloc[i, j]  = useranduser_prediction(k, df3, cosine_similarity, df4, (j, i))
df4_fix

,0,1,2,3,4,5,6,7,8,9,10,11
0,3.2,0.0,0.500000,-0.25,3.0,-1.0,-3.0,-0.333333,1.15301,-0.666667,-1.4,-3.25
1,2.2,2.0,-0.500000,-0.25,0.0,0.0,1.0,0.666667,-2.50000,-1.088235,-0.4,-0.25
2,-1.8,0.0,0.500000,-0.25,0.0,0.0,2.0,-0.333333,0.50000,0.500000,0.6,2.75
3,-3.8,-1.0,-0.565631,0.75,-3.0,-1.0,0.0,-0.565631,1.50000,0.333333,-0.4,0.00
4,0.2,-1.0,-0.500000,0.00,0.0,2.0,0.0,0.000000,0.50000,0.333333,1.6,0.75


In [14]:
lst1 = list() 
for i in range(df4_fix.shape[0]):
    lst2 = list()
    for j in range(df4_fix.shape[1]):
        lst2.append(round(df4_fix.iloc[i, j] + mean[j], 2))
    lst1.append(lst2)
result = pd.DataFrame(lst1)
result.columns = df3.columns
result.index = df3.index
result

,a,b,c,d,e,f,g,h,i,j,k,l
mathematics,9.0,6.0,7.00,5.00,8.0,5.0,4.0,7.00,5.65,5.00,5.0,2.00
chemistry,8.0,8.0,6.00,5.00,5.0,6.0,8.0,8.00,2.00,4.58,6.0,5.00
physics,4.0,6.0,7.00,5.00,5.0,6.0,9.0,7.00,5.00,6.17,7.0,8.00
literature,2.0,5.0,5.93,6.00,2.0,5.0,7.0,6.77,6.00,6.00,6.0,5.25
history,6.0,5.0,6.00,5.25,5.0,8.0,7.0,7.33,5.00,6.00,8.0,6.00


In [ ]:
## Content-based Recommendation Systems (https://machinelearningcoban.com/2017/05/17/contentbasedrecommendersys/)

In [19]:
import pandas as pd 
u_cols =  ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./u.user', sep='|', names=u_cols,encoding='latin-1')

n_users = users.shape[0]
print ('Number of users:', n_users)

Number of users: 943


In [23]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('./ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('./ua.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.to_numpy()
rate_test = ratings_test.to_numpy()

print ('Number of traing rates:', rate_train.shape[0])
print ('Number of test rates:', rate_test.shape[0])

Number of traing rates: 90570
Number of test rates: 9430


In [24]:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

items = pd.read_csv('./u.item', sep='|', names=i_cols,
 encoding='latin-1')

n_items = items.shape[0]
print ('Number of items:', n_items)

Number of items: 1682


In [26]:
X0 = items.to_numpy()
X_train_counts = X0[:, -19:]

In [27]:
#tfidf
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True, norm ='l2')
tfidf = transformer.fit_transform(X_train_counts.tolist()).toarray()

In [28]:
import numpy as np
def get_items_rated_by_user(rate_matrix, user_id):
    """
    in each line of rate_matrix, we have infor: user_id, item_id, rating (scores), time_stamp
    we care about the first three values
    return (item_ids, scores) rated by user user_id
    """
    y = rate_matrix[:,0] # all users
    # item indices rated by user_id
    # we need to +1 to user_id since in the rate_matrix, id starts from 1 
    # while index in python starts from 0
    ids = np.where(y == user_id +1)[0] 
    item_ids = rate_matrix[ids, 1] - 1 # index starts from 0 
    scores = rate_matrix[ids, 2]
    return (item_ids, scores)

In [29]:
from sklearn.linear_model import Ridge
from sklearn import linear_model

d = tfidf.shape[1] # data dimension
W = np.zeros((d, n_users))
b = np.zeros((1, n_users))

for n in range(n_users):    
    ids, scores = get_items_rated_by_user(rate_train, n)
    clf = Ridge(alpha=0.01, fit_intercept  = True)
    Xhat = tfidf[ids, :]
    
    clf.fit(Xhat, scores) 
    W[:, n] = clf.coef_
    b[0, n] = clf.intercept_

In [30]:
# predicted scores
Yhat = tfidf.dot(W) + b

In [31]:
n = 10
np.set_printoptions(precision=2) # 2 digits after . 
ids, scores = get_items_rated_by_user(rate_test, n)
Yhat[n, ids]
print('Rated movies ids :', ids )
print('True ratings     :', scores)
print('Predicted ratings:', Yhat[ids, n])

Rated movies ids : [ 37 109 110 226 424 557 722 724 731 739]
True ratings     : [3 3 4 3 4 3 5 3 3 4]
Predicted ratings: [3.18 3.13 3.42 3.09 3.35 5.2  4.01 3.35 3.42 3.72]


In [34]:
import math
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred 
        se += (e*e).sum(axis = 0)
        cnt += e.size 
    return math.sqrt(se/cnt)

print ('RMSE for training:', evaluate(Yhat, rate_train, W, b))
print ('RMSE for test    :', evaluate(Yhat, rate_test, W, b))

RMSE for training: 0.908980456282672
RMSE for test    : 1.2703282700393037


In [ ]:
## Neighborhood-Based Collaborative Filtering (https://machinelearningcoban.com/2017/05/24/collaborativefiltering/)

In [48]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

class CF(object):
    """docstring for CF"""
    def __init__(self, Y_data, k, dist_func = cosine_similarity, uuCF = 1):
        self.uuCF = uuCF # user-user (1) or item-item (0) CF
        self.Y_data = Y_data if uuCF else Y_data[:, [1, 0, 2]]
        self.k = k # number of neighbor points
        self.dist_func = dist_func
        self.Ybar_data = None
        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(self.Y_data[:, 0])) + 1
        self.n_items = int(np.max(self.Y_data[:, 1])) + 1

    def add(self, new_data):
        """
        Update Y_data matrix when new ratings come.
        For simplicity, suppose that there is no new user or item.
        """
        self.Y_data = np.concatenate((self.Y_data, new_data), axis=0)

    def normalize_Y(self):
        users = self.Y_data[:, 0]  # all users - first col of the Y_data
        self.Ybar_data = self.Y_data.copy()
        self.mu = np.zeros((self.n_users,))
        for n in range(self.n_users):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data[ids, 1]
            # and the corresponding ratings
            ratings = self.Y_data[ids, 2]
            # take mean
            m = np.mean(ratings)
            if np.isnan(m):
                m = 0  # to avoid empty array and nan value
            # normalize
            self.Ybar_data[ids, 2] = ratings - self.mu[n]

        ################################################
        # form the rating matrix as a sparse matrix. Sparsity is important
        # for both memory and computing efficiency. For example, if #user = 1M,
        # #item = 100k, then shape of the rating matrix would be (100k, 1M),
        # you may not have enough memory to store this. Then, instead, we store
        # nonzeros only, and, of course, their locations.
        self.Ybar = sparse.coo_matrix((self.Ybar_data[:, 2],
                                       (self.Ybar_data[:, 1], self.Ybar_data[:, 0])), (self.n_items, self.n_users))
        self.Ybar = self.Ybar.tocsr()

    def similarity(self):
        self.S = self.dist_func(self.Ybar.T, self.Ybar.T)

    def refresh(self):
        """
        Normalize data and calculate similarity matrix again (after
        some few ratings added)
        """
        self.normalize_Y()
        self.similarity()

    def fit(self):
        self.refresh()

    def __pred(self, u, i, normalized=1):
        """
        predict the rating of user u for item i (normalized)
        if you need the un
        """
        # Step 1: find all users who rated i
        ids = np.where(self.Y_data[:, 1] == i)[0].astype(np.int32)
        # Step 2:
        users_rated_i = (self.Y_data[ids, 0]).astype(np.int32)
        # Step 3: find similarity btw the current user and others
        # who already rated i
        sim = self.S[u, users_rated_i]
        # Step 4: find the k most similarity users
        a = np.argsort(sim)[-self.k:]
        # and the corresponding similarity levels
        nearest_s = sim[a]
        # How did each of 'near' users rated item i
        r = self.Ybar[i, users_rated_i[a]]
        if normalized:
            # add a small number, for instance, 1e-8, to avoid dividing by 0
            return (r * nearest_s)[0] / (np.abs(nearest_s).sum() + 1e-8)

        return (r * nearest_s)[0] / (np.abs(nearest_s).sum() + 1e-8) + self.mu[u]

    def pred(self, u, i, normalized=1):
        """
        predict the rating of user u for item i (normalized)
        if you need the un
        """
        if self.uuCF: return self.__pred(u, i, normalized)
        return self.__pred(i, u, normalized)

    def recommend(self, u, normalized=1):
        """
        Determine all items should be recommended for user u. (uuCF =1)
        or all users who might have interest on item u (uuCF = 0)
        The decision is made based on all i such that:
        self.pred(u, i) > 0. Suppose we are considering items which
        have not been rated by u yet.
        """
        ids = np.where(self.Y_data[:, 0] == u)[0]
        items_rated_by_u = self.Y_data[ids, 1].tolist()
        recommended_items = []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                rating = self.__pred(u, i)
                if rating > 0:
                    recommended_items.append(i)

        return recommended_items

    def print_recommendation(self):
        """
        print all items which should be recommended for each user
        """
        print('Recommendation: ')
        for u in range(self.n_users):
            recommended_items = self.recommend(u)
            if self.uuCF:
                print('    Recommend item(s):', recommended_items, 'to user', u)
            else:
                print('    Recommend item', u, 'to user(s) : ', recommended_items)


# data file
r_cols = ['user_id', 'item_id', 'rating']
ratings = pd.read_csv('ex.dat', sep = ' ', names = r_cols, encoding='latin-1')
Y_data = ratings.values

rs = CF(Y_data, k = 2, uuCF = 1)
rs.fit()

rs.print_recommendation()


rs = CF(Y_data, k = 2, uuCF = 0)
rs.fit()
rs.print_recommendation()

Recommendation: 
    Recommend item(s): [2] to user 0
    Recommend item(s): [1] to user 1
    Recommend item(s): [1] to user 2
    Recommend item(s): [2, 4] to user 3
    Recommend item(s): [1, 2, 4] to user 4
    Recommend item(s): [0, 3, 4] to user 5
    Recommend item(s): [0, 1] to user 6
Recommendation: 
    Recommend item 0 to user(s) :  [5, 6]
    Recommend item 1 to user(s) :  [1, 2, 4, 6]
    Recommend item 2 to user(s) :  [0, 3, 4]
    Recommend item 3 to user(s) :  [5]
    Recommend item 4 to user(s) :  [3, 4, 5]


In [ ]:
## Matrix Factorization Collaborative Filtering (https://machinelearningcoban.com/2017/05/31/matrixfactorization/)

In [65]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

class MF(object):
    """docstring for CF"""
    def __init__(self, Y_data, K, lam = 0.1, Xinit = None, Winit = None, 
            learning_rate = 0.5, max_iter = 1000, print_every = 100, user_based = 1):
        self.Y_raw_data = Y_data
        self.K = K
        # regularization parameter
        self.lam = lam
        # learning rate for gradient descent
        self.learning_rate = learning_rate
        # maximum number of iterations
        self.max_iter = max_iter
        # print results after print_every iterations
        self.print_every = print_every
        # user-based or item-based
        self.user_based = user_based
        # number of users, items, and ratings. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(Y_data[:, 0])) + 1 
        self.n_items = int(np.max(Y_data[:, 1])) + 1
        self.n_ratings = Y_data.shape[0]
        
        if Xinit is None: # new
            self.X = np.random.randn(self.n_items, K)
        else: # or from saved data
            self.X = Xinit 
        
        if Winit is None: 
            self.W = np.random.randn(K, self.n_users)
        else: # from daved data
            self.W = Winit
            
        # normalized data, update later in normalized_Y function
        self.Y_data_n = self.Y_raw_data.copy()


    def normalize_Y(self):
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.n_users

        # if we want to normalize based on item, just switch first two columns of data
        else: # item bas
            user_col = 1
            item_col = 0 
            n_objects = self.n_items

        users = self.Y_raw_data[:, user_col] 
        self.mu = np.zeros((n_objects,))
        for n in range(n_objects):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data_n[ids, item_col] 
            # and the corresponding ratings 
            ratings = self.Y_data_n[ids, 2]
            # take mean
            m = np.mean(ratings) 
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.mu[n] = m
            # normalize
            self.Y_data_n[ids, 2] = ratings - self.mu[n]
    def loss(self):
        L = 0 
        for i in range(self.n_ratings):
            # user, item, rating
            n, m, rate = int(self.Y_data_n[i, 0]), int(self.Y_data_n[i, 1]), self.Y_data_n[i, 2]
            L += 0.5*(rate - self.X[m, :].dot(self.W[:, n]))**2
        
        # take average
        L /= self.n_ratings
        # regularization, don't ever forget this 
        L += 0.5*self.lam*(np.linalg.norm(self.X, 'fro') + np.linalg.norm(self.W, 'fro'))
        return L
    def get_items_rated_by_user(self, user_id):
        """
        get all items which are rated by user user_id, and the corresponding ratings
        """
        ids = np.where(self.Y_data_n[:,0] == user_id)[0] 
        item_ids = self.Y_data_n[ids, 1].astype(np.int32) # indices need to be integers
        ratings = self.Y_data_n[ids, 2]
        return (item_ids, ratings)
    def get_items_rated_by_user(self, user_id):
        """
        get all items which are rated by user user_id, and the corresponding ratings
        """
        ids = np.where(self.Y_data_n[:,0] == user_id)[0] 
        item_ids = self.Y_data_n[ids, 1].astype(np.int32) # indices need to be integers
        ratings = self.Y_data_n[ids, 2]
        return (item_ids, ratings)
        
        
    def get_users_who_rate_item(self, item_id):
        """
        get all users who rated item item_id and get the corresponding ratings
        """
        ids = np.where(self.Y_data_n[:,1] == item_id)[0] 
        user_ids = self.Y_data_n[ids, 0].astype(np.int32)
        ratings = self.Y_data_n[ids, 2]
        return (user_ids, ratings)
    def updateX(self):
        for m in range(self.n_items):
            user_ids, ratings = self.get_users_who_rate_item(m)
            Wm = self.W[:, user_ids]
            # gradient
            grad_xm = -(ratings - self.X[m, :].dot(Wm)).dot(Wm.T)/self.n_ratings + \
                                               self.lam*self.X[m, :]
            self.X[m, :] -= self.learning_rate*grad_xm.reshape((self.K,))
    
    def updateW(self):
        for n in range(self.n_users):
            item_ids, ratings = self.get_items_rated_by_user(n)
            Xn = self.X[item_ids, :]
            # gradient
            grad_wn = -Xn.T.dot(ratings - Xn.dot(self.W[:, n]))/self.n_ratings + \
                        self.lam*self.W[:, n]
            self.W[:, n] -= self.learning_rate*grad_wn.reshape((self.K,))
    def fit(self):
        self.normalize_Y()
        for it in range(self.max_iter):
            self.updateX()
            self.updateW()
            if (it + 1) % self.print_every == 0:
                rmse_train = self.evaluate_RMSE(self.Y_raw_data)
                print ('iter =', it + 1, ', loss =', self.loss(), ', RMSE train =', rmse_train)
    def pred(self, u, i):
        """ 
        predict the rating of user u for item i 
        if you need the un
        """
        u = int(u)
        i = int(i)
        if self.user_based:
            bias = self.mu[u]
        else: 
            bias = self.mu[i]
        pred = self.X[i, :].dot(self.W[:, u]) + bias 
        # truncate if results are out of range [0, 5]
        if pred < 0:
            return 0 
        if pred > 5: 
            return 5 
        return pred 
        
    
    def pred_for_user(self, user_id):
        """
        predict ratings one user give all unrated items
        """
        ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
        items_rated_by_u = self.Y_data_n[ids, 1].tolist()              
        
        y_pred = self.X.dot(self.W[:, user_id]) + self.mu[user_id]
        predicted_ratings= []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                predicted_ratings.append((i, y_pred[i]))
        
        return predicted_ratings
    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0 # squared error
        for n in range(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
            SE += (pred - rate_test[n, 2])**2 

        RMSE = np.sqrt(SE/n_tests)
        return RMSE

In [66]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('./ub.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('./ub.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.to_numpy()
rate_test = ratings_test.to_numpy()

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

In [67]:
rs = MF(rate_train, K = 10, lam = .1, print_every = 10, 
    learning_rate = 0.75, max_iter = 100, user_based = 1)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print ('\nUser-based MF, RMSE =', RMSE)

iter = 10 , loss = 5.66224441409289 , RMSE train = 1.2104977438212845
iter = 20 , loss = 2.6422310682284698 , RMSE train = 1.0384547728618716
iter = 30 , loss = 1.3446056637605537 , RMSE train = 1.029620085851922
iter = 40 , loss = 0.7534182510446834 , RMSE train = 1.0292298249429979
iter = 50 , loss = 0.4824818886110831 , RMSE train = 1.0292141704017534
iter = 60 , loss = 0.3582416359775111 , RMSE train = 1.0292140764548614
iter = 70 , loss = 0.3012670247999374 , RMSE train = 1.0292142530701658
iter = 80 , loss = 0.27513927699307805 , RMSE train = 1.0292143104507783
iter = 90 , loss = 0.2631574706894393 , RMSE train = 1.0292143258428628
iter = 100 , loss = 0.2576627904137126 , RMSE train = 1.0292143297378966

User-based MF, RMSE = 1.0603799098139621


In [68]:
rs = MF(rate_train, K = 10, lam = .1, print_every = 10, learning_rate = 0.75, max_iter = 100, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print ('\nItem-based MF, RMSE =', RMSE)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


iter = 10 , loss = 5.643553179153409 , RMSE train = 1.1781704243196751
iter = 20 , loss = 2.6266648563668125 , RMSE train = 1.0057486299820975
iter = 30 , loss = 1.3283091856166849 , RMSE train = 0.9966496511102643
iter = 40 , loss = 0.736745357227294 , RMSE train = 0.9962093503964466
iter = 50 , loss = 0.4656464363653311 , RMSE train = 0.9961834493805853
iter = 60 , loss = 0.34133489839818965 , RMSE train = 0.9961812166507721
iter = 70 , loss = 0.28432846943596773 , RMSE train = 0.9961809449583123
iter = 80 , loss = 0.25818638363494306 , RMSE train = 0.9961809082867736
iter = 90 , loss = 0.24619808654888528 , RMSE train = 0.9961809039436225
iter = 100 , loss = 0.2407004617203151 , RMSE train = 0.9961809036973783

Item-based MF, RMSE = 1.0498047605125254
